In [6]:
from Mixture_Models import *

#### Simulating some data

In [7]:
#data = make_pinwheel(radial_std=0.3, tangential_std=0.05, num_classes=3,num_per_class=100, rate=0.4,rs=npr.RandomState(0))
load_data = load_csvdataset("Khan_train")
data = load_data[0]
ground_truth = load_data[3]


#### Initializing the model

In [8]:
test_GMM = GMM(data)

#### Initializing the input parameters

In [9]:
npr.seed(10)
init_params = test_GMM.init_params(num_components=3,scale=0.5,use_kmeans=True)
print(init_params)

{'log proportions': array([ 0.66579325,  0.35763949, -0.77270015]), 'means': array([[-1.55796371e-01, -1.23793101e+00, -1.35696034e-03, ...,
        -1.88622005e-01,  2.20457638e-02, -1.13153288e+00],
       [-8.94123694e-01, -2.01967210e+00,  3.37678006e-01, ...,
        -1.14804583e+00, -1.08909591e+00, -1.38331356e+00],
       [ 6.26782537e-01, -1.19743427e+00, -4.17015409e-01, ...,
        -8.21279408e-01, -7.40838082e-01, -8.66473364e-01]]), 'sqrt_covs': array([[[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 1.]],

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 1.]],

       [[1., 0., 0., ..., 0., 0., 0.],

In [21]:
'''
cluster_lls = []
for log_proportion, mean, cov_sqrt in zip(*test_GMM.unpack_params(init_params)):
    print(np.linalg.det(cov_sqrt.T @ cov_sqrt))
    #cluster_lls.append(cov_sqrt)
    cluster_lls.append(log_proportion + test_GMM.mvn_logpdf(data, mean, cov_sqrt))
#labels = np.argmax(np.array(cluster_lls).T, axis=1)
'''

1.0


FloatingPointError: overflow encountered in det

In [19]:
print(np.shape(cluster_lls[0]))

(2308, 2308)


#### Fitting the model with the above initialization parameters

In [96]:
## we are using gradient descent with momentum for 100 iterations with mass = 0.9 and learning rate = 5e-4
params_store = test_GMM.fit(init_params,"grad_descent", learning_rate = 0.0003, mass = 0.6, maxiter = 50) 

Log likelihood -49877.27108200058
Log likelihood -48235.48932577262
Log likelihood -46866.38887351064
Log likelihood -45945.03186855593
Log likelihood -45316.2048571173
Log likelihood -44862.18041313526
Log likelihood -44514.038046349
Log likelihood -44232.22693782702
Log likelihood -43993.48930751038
Log likelihood -43783.693795345
Log likelihood -43593.974437327335
Log likelihood -43418.60139544509
Log likelihood -43253.76832807692
Log likelihood -43096.87975983598
Log likelihood -42946.120479273784
Log likelihood -42800.189045747786
Log likelihood -42658.129394126794
Log likelihood -42519.22242215404
Log likelihood -42382.914964479976
Log likelihood -42248.772461608896
Log likelihood -42116.446875781134
Log likelihood -41985.654560632705
Log likelihood -41856.160722871005
Log likelihood -41727.76831382253
Log likelihood -41600.309943122316
Log likelihood -41473.64188660426
Log likelihood -41347.6395688585
Log likelihood -41222.19410127794
Log likelihood -41097.2095879985
Log likelih

#### Posthoc analysis

In [97]:
### We define an auxillary function for plotting the clusters
def draw_clusters(unpacked_params,input_data):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    for log_proportion, mean, cov_sqrt in zip(*unpacked_params):
        num_points = 100
        alpha = np.minimum(1.0, np.exp(log_proportion) * 10)
        angles = np.linspace(0, 2*np.pi, num_points)
        circle_pts = np.vstack([np.cos(angles), np.sin(angles)]).T * 2.0
        cur_pts = mean + np.dot(circle_pts, cov_sqrt)
        ax.plot(cur_pts[:, 0], cur_pts[:, 1], '-', alpha=alpha)
        ax.plot(input_data[:, 0], input_data[:, 1], 'k.')
    ax.plot(input_data[:, 0], input_data[:, 1], 'k.')

    plt.show()

In [98]:
for params in params_store:
    print("likelihood",test_GMM.likelihood(params))
    print("aic,bic",test_GMM.aic(params),test_GMM.bic(params))
    preds = test_GMM.labels(data,params)
    print(dict(zip(*np.unique(np.array(preds),return_counts=True))))
    print(100*sum(np.equal(preds,ground_truth))/len(preds))
    

    #draw_clusters(test_GMM.unpack_params(params),data)

likelihood -49877.27108200058
aic,bic 16101122.542164002 24067661.811471038


c:\Users\Admin\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\linalg\linalg.py:2158: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


{0: 20}
0.0
likelihood -48235.48932577262
aic,bic 16097838.978651546 24064378.247958582
{0: 20}
0.0
likelihood -46866.38887351064
aic,bic 16095100.777747022 24061640.047054056
{0: 20}
0.0
likelihood -45945.03186855593
aic,bic 16093258.063737111 24059797.33304415
{0: 20}
0.0
likelihood -45316.2048571173
aic,bic 16092000.409714235 24058539.67902127
{0: 20}
0.0
likelihood -44862.18041313526
aic,bic 16091092.36082627 24057631.630133305
{0: 20}
0.0
likelihood -44514.038046349
aic,bic 16090396.076092698 24056935.345399734
{0: 20}
0.0
likelihood -44232.22693782702
aic,bic 16089832.453875653 24056371.72318269
{0: 20}
0.0
likelihood -43993.48930751038
aic,bic 16089354.978615021 24055894.247922055
{0: 20}
0.0
likelihood -43783.693795345
aic,bic 16088935.38759069 24055474.656897727
{0: 20}
0.0
likelihood -43593.974437327335
aic,bic 16088555.948874654 24055095.218181692
{0: 20}
0.0
likelihood -43418.60139544509
aic,bic 16088205.20279089 24054744.472097926
{0: 20}
0.0
likelihood -43253.76832807692


In [99]:
### Final labels

np.array(test_GMM.labels(data,params_store[-1]))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)